In [2]:
import torch

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
import torch.nn as nn
import math

In [5]:
class Encoder(nn.Module):
    def __init__(self,emb_dim,num_heads,drop_prob):
        super().__init__()
        self.attention=MultiHeadAttention(emb_dim=emb_dim,num_heads=num_heads)
        self.norm1=nn.LayerNorm(emb_dim)
        self.dropout1=nn.Dropout(p=drop_prob)
        self.ffn=FeedForwardNetwork(emb_dim,hidden_dim=1028,drop_prob)
        self.norm2=nn.LayerNorm(emb_dim)
        self.dropout2=nn.Dropout(p=drop_prob)


    def forward(self,x):
        residual_x=x
        x=self.attention(x)
        x=dropout1(x)
        x=self.norm1(x+residual_x)
        residual_x=x
        x=self.ffn(x)
        x=self.dropout(x)
        x=self.norm2(x+residual_x)
        return x
        

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self,emb_dim,num_heads):
        super().__init__()                                             
        self.emb_dim=emb_dim                                # Stores dimension of each word 
        self.num_heads=num_heads                            # No of self attentions running parallel
        self.head_dim=emb_dim//num_heads                    # Each word dimension in 1 self attention
        self.qkv_layer=nn.Linear(emb_dim,3*emb_dim)         # Splitting into q,k,v (for better performance joing q,k,v[512:512:512]
        self.linear=nn.Linear(emb_dim,emb_dim)              # This layer do not change dim but improves learning after concating of q,k,v

        
    def forward(self,x):
        batch_size,each_sen_length,emb_dim=s.shape 
        # Shape of x is [batch_size,each_sen_length,emb_dim] -> {32,100,512}
        x=self.qkv(x)                                       
        # After concatenation of q,k,v-->(emb_dim+emb_dim+emb_dim) shape of x is [batch_size,each_sen_length,emb_dim*3]-> {32,100,3*512}
        x.reshape(batch_size,each_sen_length,self.num_heads,3*self.head_dim)
        # This line divides into no of self attention(num heads)  shape of x is [batch_size,each_sen_length,num_heads,(emb_dim*3)/num_heads] -> {32,100,8,3*64}
        x.permute(0,2,1,3)  
        # shape of x is [batch_size,num_heads,each_sen_length,(emb_dim*3)/num_heads] -> {32,8,100,3*64}
        query,key,value=x.chunk(3,dim=-1)
        # Dividing into q,k,v -> each dim = [batch_size,num_heads,each_sen_length,(emb_dim)/num_heads] -> {32,8,100,64}
        values=scaled_dot_attention(query,key,value)
        # dim of values = [batch_size,num_heads,each_sen_length,(emb_dim)/num_heads] -> {32,8,100,64}
        x.permute(0,1,2,3)  
        # Converting into prev dim -> shape of x is [batch_size,num_heads,each_sen_length,(emb_dim*3)/num_heads] -> {32,100,8,64}
        values.reshape(batch_size,each_sen_length,self.emd_dim)
        # dim of values = [batch_size,num_heads,each_sen_length,(emb_dim)/num_heads] -> {32,100,512}
        out=self.linear(values) 
        # After concatenation of all num_heads -> goes into liner network -> increase learning of all num_heads
        return out
        

In [ ]:
class FeedForwardNetwork(nn.Module):
    def __init__(self,emb_dim,hidden_dim,drop_prob=0.1):
        super().__init__()
        self.linear1=nn.Linear(emb_dim,hidden_dim)
        self.relu=nn.ReLU()
        self.dropout=nn.Dropout(p=drop_prob)
        self.linear2=nn.Linear(hidden_dim,emb_dim)

    def forward(self,x):
        x=self.linear1(x)
        x=self.relu(x)
        x=self.dropout(x)
        x=self.linear2(x)
        return x
        

In [9]:
def scaled_dot_attention(q,k,v):
    # dim of q/k/v = [batch_size,num_heads,each_sen_length,each_head_emb_dim(head_dim)]
    dim_k=k.size()[-1]  # dim_k = head_dim
    scaled=torch.matmul(q,k.transpose(-1,-2))/math.sqrt(dim_k)
    attention=torch.softmax(scaled)
    values=torch.matmul(attention,v)
    return values

In [6]:
tensor = torch.tensor([
    [[1, 2, 3],
     [4, 5, 6]],

    [[7, 8, 9],
     [10, 11, 12]]
]).shape

print(tensor_3d)
print(tensor_3d.shape)

tensor([[[ 1,  2,  3],
         [ 4,  5,  6]],

        [[ 7,  8,  9],
         [10, 11, 12]]])
torch.Size([2, 2, 3])


In [8]:
batch_size,each_sen_length,emb_dim=tensor.shape
print(batch_size)
print(each_sen_length)
print(emb_dim)

2
2
3


In [11]:
tensor = tensor.float()
layer=nn.Linear(emb_dim,3*emb_dim)
a=layer(tensor)

In [12]:
a.shape

torch.Size([2, 2, 9])